In [1]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [11]:
Discrete(3).sample()

2

In [15]:
Box(0,1, shape=(3,3)).sample()

array([[0.6229952 , 0.16131566, 0.75324994],
       [0.49975255, 0.9816448 , 0.32097018],
       [0.23036616, 0.9498409 , 0.38695267]], dtype=float32)

In [45]:
Dict({'height' :Discrete(2), "speed":Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([23.12915], dtype=float32))])

In [90]:
MultiBinary(4).sample()

array([1, 0, 1, 0], dtype=int8)

In [65]:
MultiDiscrete([5,2,2]).sample()

array([0, 0, 1], dtype=int64)

In [98]:
Box(low=0, high=100, shape = (1,)).sample()

array([52.43822], dtype=float32)

In [3]:
class ShowerEnv(Env) :
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low = np.array([0]), high=100, shape=(1,))
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60
        
    def step(self, action):
        # Apply temp adj
        self.state += action-1

        # Decrease showerr
        self.shower_length -=1

        # Calculate Reward
        if self.state >=37 and self.state <=39:
            reward = 1
        else :
            reward = -1

        if self.shower_length <=0:
            done = True
        else: 
            done = False
        
        info = {}

        return self.state, reward, done, info
        
    def render(self):
        # Implement viz
        pass
        
    def reset(self):
        self.state = np.array([38+random.randint(-3,3)]).astype(float)
        self.shower_length = 60
        return self.state

In [14]:
env = ShowerEnv()

In [112]:
env.observation_space.sample()

array([90.29177], dtype=float32)

In [16]:
env.reset()

array([37.])

In [18]:
episodes = 5
for episode in range(1, episodes+1) :
    obs = env.reset()
    score = 0
    over = False
    
    while not over :
        env.render()
        action = env.action_space.sample()
        obs, reward, gagal, info = env.step(action)
        score += reward
    print(f"Episode: {episode}, Score: {score}")
env.close()

KeyboardInterrupt: 

In [20]:
log_path = os.path.join('Training', 'Logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log = log_path)

Using cpu device


C:\Users\Owen\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [21]:
model.learn(total_timesteps=4000)

Logging to Training\Logs\PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -21.4    |
| time/              |          |
|    fps             | 281      |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -26.2       |
| time/                   |             |
|    fps                  | 228         |
|    iterations           | 2           |
|    time_elapsed         | 17          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011663509 |
|    clip_fraction        | 0.0428      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 0.000657    |

In [32]:
shower_path = os.path.join('Training', 'Saved Models', 'Shower_Model_PPO')

In [34]:
model.save(shower_path)

C:\Users\Owen\anaconda3\Lib\site-packages\stable_baselines3\common\save_util.py:284: UserWarning: Path 'Tutorial_RL_fromYT\Training\Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [36]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\Users\Owen\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
C:\Users\Owen\anaconda3\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
C:\Users\Owen\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:259: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(-58.6, 2.009975124224178)